In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!git clone https://github.com/Duke-BC-DL-for-Energy-Infrastructure/yolov3.git  # clone our fork of the ultralytics repo

In [ ]:
!mkdir -p data/

# get our data and unzip it
!wget -nc -O data/images.zip https://duke.box.com/shared/static/gc6g6054alnjtp0isdcccuxdr5wl59xy.zip
!wget -nc -O data/labels.zip https://duke.box.com/shared/static/nawqed2wofslp2dw5d9j17i35mmb0ul5.zip

!unzip -n -q data/images.zip -d data
!unzip -n -q data/labels.zip -d data

# get the synthetic images and labels and unzip those into the same directory
!wget -nc -O data/synthetic_images.zip https://duke.box.com/shared/static/bzvtcpzxr7xpfrfuen5w8i35wqb494z0.zip
!wget -nc -O data/synthetic_labels.zip https://duke.box.com/shared/static/svq4b2xpl46qb7xjq6ylq6sozn5v2geu.zip

!unzip -n -q data/synthetic_images.zip -d data
!unzip -n -q data/synthetic_labels.zip -d data

#get the new labelled images and labels and unzip them
!wget -nc -O data/images_new.zip https://duke.box.com/shared/static/9hc0uqxric3gptb6j92fzswcuux4ou73.zip
!wget -nc -O data/label_new.zip https://duke.box.com/shared/static/41r1grgakpjswnzu274xk8k1edo9xfkd.zip
    
!unzip -n -q data/images_new.zip -d data
!unzip -n -q data/label_new.zip -d data

In [ ]:
# delete the original data folder so we can unzip our own data folder with the wind turbine data
!rm -rf yolov3/data

# load in the data.zip file from the box
!wget -nc -O yolov3/data.zip https://duke.box.com/shared/static/mvr4grwy9ezlrlkj1fgjjtxa88iww3ew.zip

!unzip -n -q yolov3/data.zip -d yolov3

# get the synthetic_data file from the box
!wget -nc -O yolov3/data_adding_synthetic.zip https://duke.box.com/shared/static/mphxkk3h4junnij1b3u60x4lvqgjoj7z.zip

!unzip -n -q yolov3/data_adding_synthetic.zip -d yolov3

# get modified cfg file that works for 1 class
!rm -rf yolov3/cfg 

!wget -nc -O yolov3/cfg.zip https://duke.box.com/shared/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
!unzip -n -q yolov3/cfg.zip -d yolov3


In [ ]:
%pwd

In [ ]:
%cd yolov3

In [ ]:
# BEFORE RUNNING train.py: - edit the obj hyperparameter on line 29 of train.py to be 120

# Train the model with real overhead images of wind turbines
# Remember to DOWNLOAD AND SAVE YOUR WEIGHTS
!python3 train.py --cfg cfg/yolov3-spp.cfg --data data/wnd_px10_seed17.data --img-size 608 --epochs 300 --nosave --batch-size 10 --device 0

In [ ]:
# Test the model using validation set (this set is the same for both normal and for adding synthetic). Compute predictions with the very last set of weights from training
!python3 test.py --cfg cfg/yolov3-spp.cfg --data data/wnd_px10_seed17.data --img-size 608 --weights weights/last.pt --conf-thres 0.3

In [ ]:
# Train the model with real overhead images of wind turbines, but adding synthetic imagery to the training set (and not to the validation set)
# Remember to DOWNLOAD AND SAVE YOUR WEIGHTS
!python3 train.py --cfg cfg/yolov3-spp.cfg --data data_adding_synthetic/wnd_syn_real_seed17.data --img-size 608 --epochs 300 --nosave --batch-size 10 --device 0

In [ ]:
# Test the model using validation set (this set is the same for both normal and for adding synthetic). Compute predictions with the very last set of weights from training
!python3 test.py --cfg cfg/yolov3-spp.cfg --data data_adding_synthetic/wnd_syn_real_seed17.data --img-size 608 --weights weights/last.pt --conf-thres 0.3

In [ ]:
# Visualize predictions using the last.pt as weights
!python3 detect.py --names data/wnd.names --weights weights/last.pt --source ../data/images --img-size 608